# Tensores

Los tensores son una estructura de datos especializada similar a los arreglos y matrices.
En PyTorch, usamos tensores para codificar las entradas y salidas de un modelo, así como los parámetros del modelo.

Los tensores son similares a los ndarrays de [NumPy](https://numpy.org/), excepto que los tensores pueden ejecutarse en GPUs u otros aceleradores de hardware. De hecho, los tensores y los arreglos de NumPy a menudo pueden compartir la misma memoria subyacente, eliminando la necesidad de copiar datos. Los tensores también están optimizados para diferenciación automática (veremos más sobre eso más adelante en la sección [Autograd](autogradqs_tutorial.html)). Si estás familiarizado con ndarrays, te sentirás como en casa con la API de Tensores. ¡Si no, sigue adelante!

In [1]:
import torch
import numpy as np

## Inicializar un Tensor

Los tensores pueden ser inicializados de varias maneras. Echa un vistazo a los siguientes ejemplos:

### Directamente desde datos

Los tensores pueden ser creados directamente desde datos. El tipo de dato se infiere automáticamente.

In [21]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

x_data.type()

### Desde un arreglo NumPy

Los tensores pueden ser creados desde arreglos NumPy (y viceversa).

In [3]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

### Desde otro tensor:

El nuevo tensor mantiene las propiedades (forma, tipo de dato) del tensor argumento, a menos que se anule explícitamente.

In [ ]:
x_ones = torch.ones_like(x_data) # mantiene las propiedades de x_data
print(f"Tensor de Unos: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # anula el tipo de dato de x_data
print(f"Tensor Aleatorio: \n {x_rand} \n")

### Con valores aleatorios o constantes:

Para el siguiente ejemplo ``shape`` es una tupla de dimensiones del tensor. En las funciones de abajo, determina la dimensionalidad del tensor de salida.

In [ ]:
shape = (2,3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Tensor Aleatorio: \n {rand_tensor} \n")
print(f"Tensor de Unos: \n {ones_tensor} \n")
print(f"Tensor de Ceros: \n {zeros_tensor}")

---

## Atributos de un Tensor

Los atributos del tensor describen su forma, tipo de dato y el dispositivo en el que están almacenados.

In [ ]:
tensor = torch.rand(3,4)

print(f"Forma del tensor: {tensor.shape}")
print(f"Tipo de dato del tensor: {tensor.dtype}")
print(f"Dispositivo donde está almacenado el tensor: {tensor.device}")

---

## Operaciones en Tensores

PyTorch contiene más de 1200 operaciones de tensores, incluyendo aritmética, álgebra lineal, manipulación de matrices (transponer,
indexar, rebanar), muestreo y más, las cuales están descritas exhaustivamente [aquí](https://pytorch.org/docs/stable/torch.html).

Cada una de estas operaciones puede ejecutarse en la CPU y en [Acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA, o XPU. Al usar Colab, asigna un acelerador yendo a Runtime > Change runtime type > GPU.

Por defecto, los tensores se crean en la CPU. Necesitamos mover explícitamente los tensores al acelerador usando
el método ``.to`` (después de verificar la disponibilidad del acelerador). Ten en cuenta que copiar tensores grandes
entre dispositivos puede ser costoso en términos de tiempo y memoria!

In [25]:
# Movemos nuestro tensor al acelerador actual si está disponible
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

A continuación se muestran algunas de las operaciones de la lista.
Si estás familiarizado con la API de NumPy, encontrarás que la API de Tensores es muy fácil de usar.

### Indexación y rebanado (slicing) estándar tipo numpy:

In [27]:
tensor = torch.ones(4, 4)
print(f"Primera fila: {tensor[0]}")
print(f"Primera columna: {tensor[:, 0]}")
print(f"Última columna: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

### Unir tensores

Puedes usar ``torch.cat`` para concatenar una secuencia de tensores a lo largo de una dimensión dada.
Existe otro oeprador de union llamadao [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html), que es sutilmente diferente de ``torch.cat``.

In [29]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

### Operaciones aritméticas

In [ ]:
# Esto calcula la multiplicación de matrices entre dos tensores usando matmul o su función equivalente @. 
# y1, y2, y3 tendrán el mismo valor
# ``tensor.T`` devuelve la transpuesta de un tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

In [ ]:
# Esto calcula el producto elemento por elemento. z1, z2, z3 tendrán el mismo valor
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

### Tensores de un solo elemento

Si tienes un tensor de un elemento, por ejemplo despues de sumar todos los valores de un mismo tensor, puedes convertirlo a un valor numérico de Python usando ``item()``:

In [12]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

### Operaciones in-place

Las operaciones que almacenan el resultado en el mismo lugar (operando) se llaman in-place. Se denotan con un sufijo ``_``.
Por ejemplo: ``x.copy_(y)``, ``x.t_()``, cambiarán el contenido de ``x``.

In [13]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

**Nota:**
Las operaciones in-place ahorran algo de memoria, pero pueden ser problemáticas al calcular derivadas debido a una pérdida inmediata
del historial. Por lo tanto, se desaconseja su uso.

---

## Puente con NumPy

Los tensores en la CPU y los arreglos NumPy pueden compartir sus ubicaciones de memoria subyacente,  de manera que cambiar los valores de uno cambiará los valores del otro.

### Tensor a arreglo NumPy

In [14]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

Un cambio en el tensor se refleja en el arreglo NumPy.

In [15]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

### Arreglo NumPy a Tensor

In [16]:
n = np.ones(5)
t = torch.from_numpy(n)

Los cambios en el arreglo NumPy se reflejan en el tensor.

In [17]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")